# Import library

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

## Generate dummy data

In [3]:
# Number of samples
n_samples = 1000

# Repair cost ranges
damage_cost_ranges = {
    'bonnet_dent': (700000, 1000000),
    'bumper_dent': (700000, 1000000),
    'bumper_scratch': (300000, 450000),
    'car_window_damage': (300000, 900000),
    'crack': (300000, 500000),
    'door_dent': (700000, 1000000),
    'door_scratch': (300000, 450000),
    'front_windscreen_damage': (1200000, 1200000),
    'headlight_damage': (300000, 500000),
    'quarter_panel_dent': (700000, 1000000),
    'quarter_panel_scratch': (300000, 450000),
    'rear_windscreen_damage': (500000, 2000000),
    'taillight_damage': (300000, 500000),
    'tire_flat': (4000, 5000),
    'trunk_door_dent': (700000, 1000000)
}

# Define data labels / classes
data = {
    'image_id': [f'{i:03d}' for i in range(1, n_samples + 1)],
    'bonnet_dent': np.random.randint(0, 3, n_samples),  # 0-2 dents
    'bumper_dent': np.random.randint(0, 3, n_samples),  # 0-2 dents
    'bumper_scratch': np.random.randint(0, 3, n_samples),  # 0-2 scratches
    'car_window_damage': np.random.randint(0, 3, n_samples),  # 0-3 damaged
    'crack': np.random.randint(0, 5, n_samples),  # 0-4 cracks
    'door_dent': np.random.randint(0, 3, n_samples),  # 0-2 dents
    'door_scratch': np.random.randint(0, 4, n_samples),  # 0-3 scratches
    'front_windscreen_damage': np.random.randint(0, 2, n_samples),  # 0-1 damaged
    'headlight_damage': np.random.randint(0, 3, n_samples),  # 0-2 damaged
    'quarter_panel_dent': np.random.randint(0, 3, n_samples),  # 0-2 dents
    'quarter_panel_scratch': np.random.randint(0, 3, n_samples),  # 0-2 scratches
    'rear_windscreen_damage': np.random.randint(0, 2, n_samples),  # 0-1 damaged
    'taillight_damage': np.random.randint(0, 3, n_samples),  # 0-2 damaged
    'tire_flat': np.random.randint(0, 3, n_samples),  # 0-2 flat tires
    'trunk_door_dent': np.random.randint(0, 2, n_samples)  # 0-1 dents
}

# Calculate the total cost using the ranges
total_cost_min = np.zeros(n_samples)
total_cost_max = np.zeros(n_samples)
for damage, (min_cost, max_cost) in damage_cost_ranges.items():
    total_cost_min += np.array(data[damage]) * min_cost
    total_cost_max += np.array(data[damage]) * max_cost

data['total_cost_min'] = total_cost_min
data['total_cost_max'] = total_cost_max

# Create dataframe
df = pd.DataFrame(data)

df.head()

,image_id,bonnet_dent,bumper_dent,bumper_scratch,car_window_damage,crack,door_dent,door_scratch,front_windscreen_damage,headlight_damage,quarter_panel_dent,quarter_panel_scratch,rear_windscreen_damage,taillight_damage,tire_flat,trunk_door_dent,total_cost_min,total_cost_max
0,001,0,0,1,0,2,2,3,0,0,0,2,1,0,2,0,4308000.0,7710000.0
1,002,1,0,2,0,4,0,1,1,0,0,0,1,2,1,1,5804000.0,9555000.0
2,003,0,0,2,2,3,2,2,0,1,0,1,0,2,0,0,5300000.0,9050000.0
3,004,0,2,1,2,1,0,2,1,2,1,0,1,0,0,1,6900000.0,11850000.0
4,005,1,1,1,0,3,2,1,0,0,2,2,0,2,1,0,6904000.0,10305000.0


# Save dataframe as csv

In [4]:
# Save the DataFrame to a CSV file
file_path = 'repair_cost_dummy_data.csv'
df.to_csv(file_path, index=False)

print(f"Dummy CSV file created at {file_path}")

Dummy CSV file created at repair_cost_dummy_data.csv


# Read dummy csv file

In [5]:
# Load csv file
file_path = '/content/repair_cost_dummy_data.csv'
df = pd.read_csv(file_path)

In [6]:
# Take sample data
df.sample(10)

,image_id,bonnet_dent,bumper_dent,bumper_scratch,car_window_damage,crack,door_dent,door_scratch,front_windscreen_damage,headlight_damage,quarter_panel_dent,quarter_panel_scratch,rear_windscreen_damage,taillight_damage,tire_flat,trunk_door_dent,total_cost_min,total_cost_max
999,1000,0,1,0,2,0,1,2,0,0,0,2,0,2,0,1,4500000.0,7600000.0
732,733,0,0,0,1,2,0,0,0,0,0,2,1,0,0,1,2700000.0,5800000.0
573,574,0,0,0,0,0,1,0,1,0,2,0,0,0,2,0,3308000.0,4210000.0
142,143,2,0,1,2,3,0,0,1,1,2,2,0,1,1,0,7004000.0,10855000.0
575,576,1,2,0,1,4,0,2,1,0,1,1,1,1,0,0,7200000.0,11950000.0
454,455,0,0,2,1,1,1,1,1,0,2,0,1,2,0,0,5900000.0,9950000.0
594,595,0,2,0,0,2,2,3,0,1,0,0,0,0,1,1,5304000.0,7855000.0
785,786,1,2,0,2,3,2,1,1,1,1,2,0,1,0,1,9100000.0,13850000.0
252,253,1,1,2,1,1,0,1,0,2,2,1,1,2,0,1,7000000.0,12200000.0
691,692,1,1,1,0,4,0,0,0,0,2,1,0,0,1,0,4604000.0,6905000.0


In [7]:
# Define features and target
X = df.drop(['image_id', 'total_cost_min', 'total_cost_max'], axis=1)

# Combine min and max cost into one target variable
y = np.column_stack((df['total_cost_min'], (df['total_cost_max'])))

In [8]:
X.head()

,bonnet_dent,bumper_dent,bumper_scratch,car_window_damage,crack,door_dent,door_scratch,front_windscreen_damage,headlight_damage,quarter_panel_dent,quarter_panel_scratch,rear_windscreen_damage,taillight_damage,tire_flat,trunk_door_dent
0,0,0,1,0,2,2,3,0,0,0,2,1,0,2,0
1,1,0,2,0,4,0,1,1,0,0,0,1,2,1,1
2,0,0,2,2,3,2,2,0,1,0,1,0,2,0,0
3,0,2,1,2,1,0,2,1,2,1,0,1,0,0,1
4,1,1,1,0,3,2,1,0,0,2,2,0,2,1,0


In [9]:
y

array([[ 4308000.,  7710000.],
       [ 5804000.,  9555000.],
       [ 5300000.,  9050000.],
       ...,
       [ 4908000.,  7210000.],
       [ 8108000., 11860000.],
       [ 4500000.,  7600000.]])

# Normalize and scale features

In [10]:
# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [11]:
# Scale the target variable
y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y)

In [12]:
# Split the data to train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
import joblib
import json

# Extract parameters for JSON conversion
scaler_params = {
    'mean': scaler.mean_.tolist(),
    'std': scaler.scale_.tolist()
}

y_scaler_params = {
    'min': y_scaler.data_min_.tolist(),
    'max': y_scaler.data_max_.tolist()
}



# Save parameters to JSON
with open('scaler.json', 'w') as f:
    json.dump(scaler_params, f)

with open('y_scaler.json', 'w') as f:
    json.dump(y_scaler_params, f)


## Create Model

In [14]:
# The model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, input_shape=[X_train.shape[1]]),
    tf.keras.layers.Dense(2)
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, validation_split=0.2, epochs=200, batch_size=32)

Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 2.7929 - mae: 1.3266 - val_loss: 2.0841 - val_mae: 1.1711
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.8963 - mae: 1.0873 - val_loss: 1.4416 - val_mae: 0.9722
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2552 - mae: 0.8872 - val_loss: 0.9925 - val_mae: 0.8062
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8059 - mae: 0.7213 - val_loss: 0.6930 - val_mae: 0.6736
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5970 - mae: 0.6164 - val_loss: 0.4762 - val_mae: 0.5584
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3941 - mae: 0.5003 - val_loss: 0.3282 - val_mae: 0.4635
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2954 - mae: 0.4327 - val_loss: 0.2247 - val_mae: 0.3837
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1950 - mae: 0.3564 - val_loss: 0.1533 - val_mae: 0.3166
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1330 - mae: 0.

# Evaluate the model

In [15]:
# Evaluate model performance
loss, mae = model.evaluate(X_test, y_test)
print("Mean Absolute Error:", mae)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.7320e-14 - mae: 1.3407e-07  
Mean Absolute Error: 1.3430137357772765e-07


In [16]:
# Predicting model
predictions = model.predict(X_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


# Test using new dummy data

In [17]:
# Predicting total cost for new data

new_data = pd.DataFrame({
    'bonnet_dent': [0, 0],
    'bumper_dent': [0, 0],
    'bumper_scratch': [0, 0],
    'car_window_damage': [0, 1],
    'crack': [0, 0],
    'door_dent': [3, 0],
    'door_scratch': [0, 0],
    'front_windscreen_damage': [0, 0],
    'headlight_damage': [0, 0],
    'quarter_panel_dent': [1, 0],
    'quarter_panel_scratch': [0, 0],
    'rear_windscreen_damage': [0, 0],
    'taillight_damage': [0, 0],
    'tire_flat': [0, 0],
    'trunk_door_dent': [0, 0]
})

In [18]:
# Standardize the new data
new_data_scaled = scaler.transform(new_data)

In [19]:
# Prediction results
predicted_costs_scaled = model.predict(new_data_scaled)
predicted_costs = y_scaler.inverse_transform(predicted_costs_scaled)

for min_cost, max_cost in predicted_costs:
    print(f'Predicted Cost Range: Rp{int(min_cost)} - Rp{int(max_cost)}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted Cost Range: Rp2799997 - Rp3999997
Predicted Cost Range: Rp299998 - Rp899999


# Convert Model to TensorFlow.js


In [20]:
# Menyimpan model ke format SavedModel
model.save('/content/my_saved_model.h5')

In [21]:
!pip install tensorflowjs

In [ ]:
!tensorflowjs_converter \
  --input_format=tf_saved_model \
  --output_format=tfjs_graph_model \
  /content/my_saved_model \
  /content/my_tfjs_model
